In [23]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from tslearn.clustering import silhouette_score
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd
import numpy as np

reviews = pd.read_excel('bookings reviews.xlsx')
reviews['date'] = pd.to_datetime(reviews['date'])
reviews = reviews.sort_values(by='date').reset_index(drop=True)
residences = reviews['property_name'].value_counts().index.to_list()

def get_sma_ratings(res):
    res_df = reviews.copy()
    res_df = res_df[res_df['property_name']==res]
    notes = res_df.copy().iloc[:,-9:]
    notes.index = res_df.copy()['date']
    notes = notes[['Value for money','Cleanliness','Facilities','Location','Comfort', 'Staff']].dropna()
    daily_mean = notes.resample('D').mean().dropna()
    smoothed = daily_mean.rolling(window=30).mean()
    smoothed['residence'] = res
    return smoothed

df = pd.DataFrame()
for res in residences:
    df = pd.concat([df,get_sma_ratings(res)], axis=0)
df = df.dropna()

In [24]:
corpus = reviews['review_text_negative'].dropna().to_list()
pos_hypotheses = [
    "Ce commentaire valorise explicitement la propreté de l'appartement",
    "Ce commentaire valorise explicitement le confort dans l'appartement",
    "Ce commentaire valorise explicitement l'emplacement de l'appartement",
    "Ce commentaire valorise explicitement le process de check-in check-out",
    "Ce commentaire valorise explicitement le calme dans l'appartement",
    "Ce commentaire valorise explicitement les équipements fournis dans l'appartement",
    "Ce commentaire valorise explicitement l'interaction avec le staff",
    "Ce commentaire valorise explicitement le rapport qualité / prix de l'appartement",
    "Ce commentaire valorise explicitement la sécurité de l'appartement",
    "Ce commentaire valorise explicitement le design / décoration de l'appartement",
    "Ce commentaire valorise explicitement la luminosité au sein de l'appartement",
    "Ce commentaire est négatif",
    "Ce commentaire n'est pas très clair"
]

neg_hypotheses = [
    "La propreté de l'appartement est explicitement critiquée.",
    "L'appartement présente explicitement des problèmes d'odeurs.",
    "Des problèmes de nuisance sonore sont explicitement mentionnés.",
    "Des équipements de cuisine défectueux ou manquants sont explicitement mentionnés.",
    "La communication ou la gestion du personnel est explicitement mentionnée.",
    "Des problèmes liés à la plomberie ou à l'eau chaude sont explicitement mentionnés.",
    "Le commentaire mentionne explicitement un problème avec la literie ou le nombre de lits.",
    "Des problèmes concernant l'éclairage de l'appartement sont explicitement mentionnées.",
    "Le caractère impersonnel ou l'automatisation excessive de l'appartement sont explicitement mentionnés.",
    "L'appartement ne ressemble explicitement pas à sa représentation en ligne.",
    "Des problèmes explicitement liés à la sécurité ou à l'accès de l'appartement sont mentionnés.",
    "Des problèmes d'accessibilité sont explicitement mentionnés.",
    "Des problèmes de communication avec le staff sont explicitement mentionnés.",
    "Des problèmes de stationnement automobile sont explicitement mentionnés.",
    "Le rapport qualité-prix est explicitement mentionné comme étant problématique.",
    "Des problèmes spécifiques avec des équipements de l'appartement (hors cuisine) sont explicitement mentionnés."
]

In [25]:
positive = pd.read_excel('positive reviews.xlsx').iloc[:,1:]
positive.rename(columns={positive.columns[0]: 'review_text_positive'}, inplace=True)
positive.drop_duplicates(inplace=True)
pos_reviews = pd.merge(reviews, positive, on='review_text_positive', how='left')
pos_classes = pos_reviews.columns.to_list()[15:]

In [26]:
negative = pd.read_excel('negative reviews.xlsx').iloc[:,1:]
negative.rename(columns={positive.columns[0]: 'review_text_negative'}, inplace=True)
negative['review_text_negative'] = corpus
negative.drop_duplicates(inplace=True)
cols = negative.columns.to_list()
ordered_cols = cols[-1:] + cols[:-1]
negative = negative[ordered_cols]
negative_reviews = pd.merge(reviews, negative, on='review_text_negative', how='left')
neg_classes = negative_reviews.columns.to_list()[15:]

In [27]:
non_na_negative_reviews = negative_reviews[~negative_reviews[neg_classes].isna().all(axis=1)]

In [28]:
non_na_positive_reviews = pos_reviews[~pos_reviews[pos_classes].isna().all(axis=1)]

In [29]:
def get_rev_sum(res,pos):
    if pos==True:
        dd = non_na_positive_reviews
        classes = pos_classes
    else:
        dd = non_na_negative_reviews
        classes = neg_classes
    res_df = dd.copy()
    res_df = res_df[res_df['property_name']==res]
    notes = res_df.copy()[classes]
    notes.index = res_df.copy()['date']
    daily_sum = notes.resample('D').sum().dropna()
    smoothed = daily_sum.rolling(window=30).sum()
    smoothed['residence'] = res
    return smoothed

In [327]:
res = residences[0]

In [32]:
init_pos=pd.DataFrame()
init_neg=pd.DataFrame()
for res in residences:
    init_pos = pd.concat([init_pos,get_rev_sum(res,pos=True)],axis=0)
    init_neg = pd.concat([init_neg,get_rev_sum(res,pos=False)],axis=0)

In [33]:
init_pos.to_excel('rollingsum_positive.xlsx')
init_neg.to_excel('rollingsum_negative.xlsx')

In [175]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_name = "joeddav/xlm-roberta-large-xnli"
access_token = "XXXXXXXXXXXXXXXXXX"

device = torch.device("cuda")
nli_model = AutoModelForSequenceClassification.from_pretrained(model_name, use_auth_token=access_token).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=access_token)

def zero_shot_classification_xlm_roberta(sequence, hypotheses, device='cuda', threshold=0.95):
    with torch.no_grad():
        inputs = tokenizer([sequence] * len(hypotheses), hypotheses, return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        logits = nli_model(**inputs)[0]

        entail_contradiction_logits = logits[:, [0, 2]]
        probs = entail_contradiction_logits.softmax(dim=1)
        probs_for_entailment = probs[:, 1].detach().cpu().numpy()  # Convert to numpy array

        del inputs

    filtered_results = {hypothesis: prob for hypothesis, prob in zip(hypotheses, probs_for_entailment) if prob > threshold}

    return filtered_results

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [230]:
results = []

for description in tqdm(corpus[:]):
    if len(description.split()) > 500: 
        continue
    result = zero_shot_classification_xlm_roberta(description, neg_hypotheses)
    results.append([description, result])
    torch.cuda.empty_cache()

100%|██████████| 13785/13785 [46:12<00:00,  4.97it/s] 


In [ ]:
results

In [231]:
import pandas as pd

rows = []
for review, classes in results:
    row_data = {'Review': review}
    row_data.update({label: 1 if label in classes else 0 for label in neg_hypotheses})
    rows.append(row_data)

df = pd.DataFrame(rows, columns=['Review'] + neg_hypotheses)
df.tail()

,Review,La propreté de l'appartement est explicitement critiquée.,L'appartement présente explicitement des problèmes d'odeurs.,Des problèmes de nuisance sonore sont explicitement mentionnés.,Des équipements de cuisine défectueux ou manquants sont explicitement mentionnés.,La communication ou la gestion du personnel est explicitement mentionnée.,Des problèmes liés à la plomberie ou à l'eau chaude sont explicitement mentionnés.,Le commentaire mentionne explicitement un problème avec la literie ou le nombre de lits.,Des problèmes concernant l'éclairage de l'appartement sont explicitement mentionnées.,Le caractère impersonnel ou l'automatisation excessive de l'appartement sont explicitement mentionnés.,L'appartement ne ressemble explicitement pas à sa représentation en ligne.,Des problèmes explicitement liés à la sécurité ou à l'accès de l'appartement sont mentionnés.,Des problèmes d'accessibilité sont explicitement mentionnés.,Des problèmes de communication avec le staff sont explicitement mentionnés.,Des problèmes de stationnement automobile sont explicitement mentionnés.,Le rapport qualité-prix est explicitement mentionné comme étant problématique.,Des problèmes spécifiques avec des équipements de l'appartement (hors cuisine) sont explicitement mentionnés.
13780,De buurt is wat guur in de avond; veel zwerver...,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
13781,A fuir ! Seconde fois en 6 mois et le même con...,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
13782,"Geen propere kamer, geen propere handdoeken. T...",1,1,1,1,0,0,0,1,0,1,0,0,0,0,1,1
13783,No hot water duing our stay for 5 days and whe...,1,1,1,0,0,0,0,1,0,0,0,1,1,0,1,0
13784,"Pas d'eau chaude pour se doucher, pas de téléc...",1,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1


In [232]:
df.to_excel('negative reviews.xlsx')

################################################################# LLM CLUSTERING ###################################################################

In [ ]:
def get_elbow_curve(df, min_clusters, max_clusters):
    inertia = np.array([])
    silhouette= np.array([])
    for k in tqdm(range(min_clusters, max_clusters)):
        KM = KMeans(n_clusters=k, n_init=10, random_state=8).fit(df)
        inertia = np.append(inertia, KM.inertia_)
        silhouette = np.append(silhouette, silhouette_score(df, KM.labels_, metric='euclidean'))
    plt.plot(range(min_clusters,max_clusters),inertia)
    plt.show()
    plt.plot(range(min_clusters,max_clusters),silhouette)
    plt.show()
    return inertia, silhouette

def get_labels(df, n_clust):
    cluster_labels = KMeans(n_clusters=n_clust, n_init=10, random_state = 8).fit(df)#.iloc[:,-lookback:])
    print(f"inertia : {silhouette_score(df, cluster_labels.labels_, metric='euclidean')}")
    cluster_labels = pd.Series(cluster_labels.labels_, index = df.index)
    return cluster_labels

def pca_transform(array, n_components):
    pca = PCA(n_components = n_components)
    pca.fit(array)
    eig_val, eig_ratio = pca.explained_variance_, pca.explained_variance_ratio_
    print(f"Actual E.V. ratio : {np.round(eig_ratio.cumsum()[-1:][0]*100,2)}")
    return pca.transform(array)

def get_labels(df, n_clust):
    cluster_labels = KMeans(n_clusters=n_clust, n_init=10, random_state = 8).fit(df)
    return cluster_labels.labels_

In [88]:
model = SentenceTransformer(r"sentence-transformers/distiluse-base-multilingual-cased", device='cuda')
#'all-MiniLM-L6-v2'
embeddings = model.encode(corpus)
pca_embeddings = pca_transform(embeddings, 383)

Actual E.V. ratio : 99.49


In [ ]:
import matplotlib.pyplot as plt
get_elbow_curve(pca_embeddings, 8, 25)

In [89]:
n_clusters = 21
labels = get_labels(pca_embeddings,n_clusters)

In [90]:
from sklearn.neighbors import NearestNeighbors
def compute_embedding_centroids(embeddings, cluster_number):
    return np.mean(embeddings[np.where(labels==cluster_number)],axis=0)

In [91]:
nn_model = NearestNeighbors(n_neighbors=5)
nn_model.fit(embeddings);

In [92]:
def get_nn(embeddings, n_clust):
    centroid = compute_embedding_centroids(embeddings, n_clust)
    dists, indexs = nn_model.kneighbors([centroid])
    print(dists)
    return np.array(corpus)[indexs.tolist()].tolist()